## Imports

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, ToPILImage, CenterCrop, RandomResizedCrop
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, resnet18, inception_v3

from torchvision.models.alexnet import AlexNet_Weights
from torchvision.models.inception import Inception_V3_Weights
from torchvision.models.resnet import ResNet18_Weights

data_path = "../data/"

## Data preparation

In [3]:
from download_rps import download_rps

train_data_path = data_path + "rps"
val_data_path = data_path + "rps-test-set"
download_rps(data_path)

rps folder already exists!
rps-test-set folder already exists!


In [4]:
# ImageNet statistics
normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transformer = Compose([Resize(256), CenterCrop(224), ToTensor(), normalizer])

#transformer = ResNet18_Weights.IMAGENET1K_V1.transforms

train_data = ImageFolder(root=train_data_path, transform=transformer)
val_data = ImageFolder(root=val_data_path, transform=transformer)

# Builds a loader of each set
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

## Feature extraction

### Model configuration

In [5]:
def freeze_model(model):
    for parameter in model.parameters():
        parameter.requires_grad = False

# Set the seed
torch.manual_seed(42)

# Load the model
resnet = resnet18(weights=ResNet18_Weights.DEFAULT)

# Change the top layer to Identity
resnet.fc = nn.Identity()
# Freeze the model
freeze_model(resnet)

### Preprocess data

In [37]:
def preprocess_dataset(model, dataset):
    features = torch.Tensor()
    labels = torch.Tensor()
    for x, y in dataset:
        model.eval()
        features = torch.cat([features, model(x)])
        labels = torch.cat([labels, y])

    dataset = TensorDataset(features, labels)
    return dataset

# Preprocess the data
train_preproc = preprocess_dataset(resnet, train_loader)
val_preproc = preprocess_dataset(resnet, val_loader)

### Save features

In [8]:
train_preproc_path = data_path + "train_preproc.pth"
val_preproc_path = data_path + "val_preproc.pth"
torch.save(train_preproc.tensors, train_preproc_path)
torch.save(val_preproc.tensors, val_preproc_path)

### Load features

In [9]:
train_preproc_path = data_path + "train_preproc.pth"
val_preproc_path = data_path + "val_preproc.pth"
train_preproc_data = TensorDataset(*torch.load(train_preproc_path))
val_preproc_data = TensorDataset(*torch.load(val_preproc_path))
train_preproc_loader = DataLoader(train_preproc_data, batch_size=16, shuffle=True)
val_preproc_loader = DataLoader(val_preproc_data, batch_size=16)

## Top model

### Model configuration

In [10]:
torch.manual_seed(42)
top_model = nn.Sequential(nn.Linear(512, 3))
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_top = optim.Adam(top_model.parameters(), lr=3e-4)

### Model training

In [11]:
from stepbystep import StepByStep

sbs_top = StepByStep(top_model, multi_loss_fn, optimizer_top)
sbs_top.set_loaders(train_preproc_loader, val_preproc_loader)
sbs_top.train(10)

### Evaluation

In [17]:
StepByStep.loader_apply(val_preproc_loader, sbs_top.correct)

tensor([[ 94, 124],
        [124, 124],
        [100, 124]])

## Using the original dataset

### Reattach the top model

In [13]:
resnet.fc = top_model
sbs_temp = StepByStep(resnet, None, None)

### Evaluate

In [14]:
StepByStep.loader_apply(val_loader, sbs_temp.correct)

tensor([[ 94, 124],
        [124, 124],
        [100, 124]])